In [21]:
%pip install -q roboflow
from roboflow import Roboflow

rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [23]:
rf = Roboflow(api_key="")
project = rf.workspace().project("vehicles-openimages-o42qs")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Vehicles-OpenImages-1 in yolov5pytorch:: 100%|██████████| 1265/1265 [00:00<00:00, 1470.72it/s]


In [24]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14486, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 14486 (delta 18), reused 30 (delta 11), pack-reused 14446
Receiving objects: 100% (14486/14486), 13.47 MiB | 31.57 MiB/s, done.
Resolving deltas: 100% (9991/9991), done.
/content/yolov5/yolov5


In [27]:
!python train.py --img 416 --batch 16 --epochs 100 --data Vehicles-OpenImages-1/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=Vehicles-OpenImages-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-226-gfde7758 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hs

In [28]:
train_n = ''

!python detect.py --weights runs/train/exp{train_n}/weights/best.pt --img 416 --save-txt --conf 0.5 --source Vehicles-OpenImages-1/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=Vehicles-OpenImages-1/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-226-gfde7758 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/63 /content/yolov5/yolov5/Vehicles-OpenImages-1/test/images/00dea1edf14f09ab_jpg.rf.5015774ac0e457a71a7487abaf231868.jpg: 416x416 1 Ambulance, 8.2ms
image 2/63 /content/yolov5/yolov5/Vehicles-OpenImages-1/test/images/00e481ea1a520175_jpg.rf.df95e719f040ca9163da366f9e6747c1.jpg: 416x416 3 Buss, 8.2ms
image 3/63 /content/yolov

In [29]:
classes = {0: 'Ambulance',
           1: 'Bus',
           2: 'Car',
           3: 'Motorcycle',
           4: 'Truck'}

In [30]:
import pandas as pd
import os

df = pd.DataFrame(columns=["objects"])

det_n = ''
dir = f"runs/detect/exp{det_n}/labels"
for filename in os.listdir(dir):
  with open(os.path.join(dir, filename), "r") as f:
    text = f.readlines()
    objects = []
    for i in text:
      label = int(i.split(' ')[0])
      objects.append(classes[label])
      index = filename[:-3]+"jpg"
      df.loc[index, 'objects'] = objects

df.head()


,objects
c0a105b4fe3a7c55_jpg.rf.27de9e6de9a5c926a6edeb77f8df3d3d.jpg,[Bus]
b52533fcc735c023_jpg.rf.018f6086f128fde635eabdf05d1a8fb4.jpg,[Ambulance]
a7674f15d9b1f35f_jpg.rf.d4e86f09cd026441ac5ea90d28ee9f33.jpg,[Ambulance]
479012386aa308fb_jpg.rf.6af894f62ffc7f7d84d4c0513a493796.jpg,[Bus]
b3736bcf6bec5308_jpg.rf.ef056de5fe22574db65b3be6f53a29e0.jpg,[Bus]


In [31]:
df.to_csv("/content/result.csv")

In [32]:
def encode(image, objects):
  for x in objects:
    data.loc[image, x] = True

data = pd.DataFrame(index = df.index, columns = classes.values())

for index, row in df.iterrows():
  encode(index, row['objects'])
data = data.fillna(False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, c0a105b4fe3a7c55_jpg.rf.27de9e6de9a5c926a6edeb77f8df3d3d.jpg to 59b1c028bd8b08dc_jpg.rf.e09c4fb4fe8d8d1dc487c6a0ef9f4d9f.jpg
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Ambulance   57 non-null     bool 
 1   Bus         57 non-null     bool 
 2   Car         57 non-null     bool 
 3   Motorcycle  57 non-null     bool 
 4   Truck       57 non-null     bool 
dtypes: bool(5)
memory usage: 2.8+ KB


In [33]:
from mlxtend.frequent_patterns import apriori

min_support = 2/len(data.index)
itemsets = apriori(data, 
                   min_support = min_support,
                   max_len = 2,
                   use_colnames = True)

itemsets.head()

,support,itemsets
0,0.192982,(Ambulance)
1,0.210526,(Bus)
2,0.491228,(Car)
3,0.122807,(Motorcycle)
4,0.105263,(Truck)


In [34]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets, min_threshold=min_support)

rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)[0])
rules["consequents"] = rules["consequents"].apply(lambda x: list(x)[0])

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Ambulance,Car,0.192982,0.491228,0.035088,0.181818,0.370130,-0.059711,0.621832
1,Car,Ambulance,0.491228,0.192982,0.035088,0.071429,0.370130,-0.059711,0.869096
2,Bus,Car,0.210526,0.491228,0.035088,0.166667,0.339286,-0.068329,0.610526
3,Car,Bus,0.491228,0.210526,0.035088,0.071429,0.339286,-0.068329,0.850202


In [35]:
rels = rules[rules['lift'] > 1]
rels = rels[['antecedents', 'consequents']]

rels.head()

,antecedents,consequents


In [36]:
!pip install Cython
!pip install owlready2
from owlready2 import *

onto = get_ontology('http://onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import types

with onto:
  onto_cl = types.new_class('Piece', (Thing,))
  for i, cl in enumerate(classes.values()):
    inst = onto_cl(f'piece_{i}')
    inst.label = cl

onto.save("/content/onto.owl")

In [38]:
with onto:
  types.new_class('isAssociatedWith', (ObjectProperty, TransitiveProperty))
  for index, rel in rels.iterrows():
    onto_ant = onto.search_one(label=rel['antecedents'])
    onto_con = onto.search_one(label=rel['consequents'])
    onto_ant.isAssociatedWith.append(onto_con)
    
onto.save("/content/onto.owl")

In [39]:
with onto:
  sync_reasoner_pellet(infer_property_values = True)

onto.save('onto_reasonde.owl', format = 'ntriples')


* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/p